In [36]:
#Анализ данных по ковиду

#таблица со случаями в странах
import pandas as pd
#https://www.kaggle.com/datasets/sudalairajkumar/novel-corona-virus-2019-dataset - data source
covid_data = pd.read_csv('data/covid_data.csv')
#display(covid_data.shape) (306429, 6) 

#таблица о вакцинах против Ковид по странам
#https://www.kaggle.com/datasets/gpreda/covid-world-vaccination-progress - data source
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']]
#print(vaccinations_data)

covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()
print(covid_data.shape) #(86785, 5)
covid_data['date'] = pd.to_datetime(covid_data['date'])
#print(covid_data[covid_data['country'] == 'China'])
#print(covid_data[covid_data['country'] == 'Albania']['date'].max())

#предобработка данных
# Группируем таблицу по дате и названию страны 
# и рассчитываем суммарные показатели по всем регионам. 
# Тем самым переходим от данных по регионам к данным по странам:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

#Преобразуем даты в формат datetime с помощью функции pd.to_datetime():
covid_data['date'] = pd.to_datetime(covid_data['date'])

#Создадим признак больных на данный момент (active). 
# Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

#Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. 
# Для этого отсортируем данные по названиям стран, а затем по датам. 
# После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

#В таблице vaccinations_data преобразовать столбцы в формат datetime:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

#Tasks

#столбцы в таблицах
#print(covid_data.columns) #'date', 'country', 'confirmed', 'deaths', 'recovered', 'active', 'daily_confirmed', 'daily_deaths', 'daily_recovered'
#print(vaccinations_data.columns) #'country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_vaccinated_per_hundred', 'people_fully_vaccinated', 
#'people_fully_vaccinated_per_hundred', 'daily_vaccinations','vaccines'

# За какой период представлены данные в таблице covid_data? 
# В качестве ответа введите даты в формате datetime (без указания времени).
# Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД
#print(covid_data['date'].max())
# print(f"Данные о заболеваемости предоставлены от {covid_data['date'].min()} до  {covid_data['date'].max()}")
#print(covid_data['date'].min())

#За какой период представлены данные в таблице vaccinations_data? 
# В качестве ответа введите даты в формате datetime без указания времени.
# Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД
#print(vaccinations_data['date'].min())
#print(vaccinations_data['date'].max())
# print(f"Данные о вакцинации предоставлены от {vaccinations_data['date'].min()} до  {vaccinations_data['date'].max()}")

#С помощью метода merge() объедините таблицы covid_data и vaccinations_data по столбцам date и country.
# Тип объединения выставьте так, чтобы в результирующую таблицу попали только наблюдения за период, 
# вычисленный в задании 3.1. 
# То есть в результирующую таблицу должны попасть все записи из таблицы covid_data 
# и из её пересечения с vaccinations_data, но не более. 
# Результат объединения занесите в переменную covid_df
covid_df = covid_data.merge(vaccinations_data, on=['date', 'country'], how='left')
#print(covid_data.shape) 86785, 9
#print(vaccinations_data.shape) 42795, 9
#print(covid_df.shape) 86785, 16 - ок, т тк остались все строки covid_data и все столбцы объединились, за исключением 2 общих (не 18, а 16 стало)
#print(f'Данные в объединенной таблице представлены с {covid_df['date'].min()} до {covid_df['date'].max()}')

#Сколько строк и столбцов в таблице covid_df?
# Введите ответ в виде двух чисел через дефис (например, 333-33): 
# первое число — количество строк, второе число — количество столбцов.
#print(covid_df.shape)

#В таблице covid_df создайте признаки death_rate — общий процент смертей среди зафиксированных случаев (летальность)
# и recover_rate — общий процент случаев выздоровления. 
# Данные характеристики рассчитайте как отношение числа смертей (deaths) и числа выздоровлений (recovered) 
# к числу зафиксированных случаев (confirmed) и умножьте результаты на 100%.
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed']) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed']) * 100

#Какова максимальная летальность в США (United States) за весь период? 
# Ответ округлите до второго знака после запятой - 7.19
#print(round(covid_df[covid_df['country'] == 'United States']['death_rate'].max(), 2))

#Чему равен средний процент выздоровевших в России (Russia)? Ответ округлите до второго знака после запятой - 67.06
#print(round(covid_df[covid_df['country'] == 'Russia']['recover_rate'].mean(), 2)) 


#Визуализация!!!

#как менялось ежедневное число заболевших (daily_confirmed) во всём мире во времени. 
# Cгруппируем таблицу по датам и подсчитаем суммарное число зафиксированных случаев по дням.
# Строим график с помощью метода plot(). Будем использовать линейный график размером 12x4. 
# Подпишем график и отобразим сетку. Параметр lw (line width) отвечает за ширину линии для линейного графика.
import matplotlib.pyplot as plt
grouped_cases = covid_df.groupby('date')['daily_confirmed'].sum()
#grouped_cases.plot(
    #kind='line',
    #figsize=(12, 4),
    #title='Ежедневная заболеваемость во времени',
    #grid = True,
    #lw=3
#)

#построим гистограмму, которая покажет распределение ежедневной заболеваемости во всём мире. 
# Для этого параметр kind выставляем на значение 'hist'. 
# Параметр bins (корзины) отвечает за число прямоугольников в гистограмме — пусть их будет 10.
#grouped_cases.plot(
#    kind='hist',
#    figsize=(10, 6),
#    title='Распределение ежедневной заболеваемости',
#    grid = True,
#    color = 'black',
#    bins=10

#построим столбчатую диаграмму, которая покажет ТОП-10 стран по суммарной заболеваемости.
# Для этого сгруппируем данные по странам и вычислим последний зафиксированный показатель 
# с помощью агрегирующего метода last() — он возвращает последнее значение в столбце DataFrame.
# Для построения столбчатой диаграммы значение параметра kind выставляем на 'bar'. 
# Параметр colormap отвечает за цветовую гамму графика.
grouped_country = covid_df.groupby(['country'])['confirmed'].last()
grouped_country = grouped_country.nlargest(10)
#grouped_country.plot(
#    kind='bar',
#    grid=True,
#    figsize=(12, 4),
#    colormap='plasma'
#)

#посмотрим, как в этих десяти странах соотносится количество заболевших и умерших от вируса. 
# Для этого отобразим сразу два показателя на столбчатой диаграмме: 
grouped_country = covid_df.groupby(['country'])[['confirmed', 'deaths']].last()
grouped_country = grouped_country.nlargest(10, columns=['confirmed'])
#grouped_country.plot(kind='bar', grid=True, figsize=(12, 4))

#Столбчатая диаграмма для пяти стран с наименьшим общим числом вакцинаций 
# на последний день рассматриваемого периода (total_vaccinations)
#covid_df.groupby(['country'])['total_vaccinations'].last().nsmallest(5).plot(kind='bar')


#matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#диаграмма рассеяния, которая покажет зависимость числа ежедневно обнаруживаемых случаев заражения (daily_confirmed) 
# от общего количества привитых вторым компонентом вакцины (people_fully_vaccinated) в США.

us_data = covid_df[covid_df['country'] == 'United States']
"""
fig = plt.figure(figsize=(8, 4))
axes = fig.add_axes([0, 0, 1, 1])
axes.scatter(
    x=us_data['people_fully_vaccinated'], 
    y=us_data['daily_confirmed'], 
    s=100,
    marker='o',
    c = 'blue'
)
"""
#Ojo
# Обратите внимание, что диаграмма показывает соотношение 
# только для десяти популярных комбинаций вакцин, 
# а не для всех вакцин в совокупности, что является не совсем корректным для круговой диаграммы, 
# так как она должна отображать единое целое.
# К тому же наша круговая диаграмма считает количество использований каждой комбинации в таблице, 
# а не количество стран, в которых используется эта комбинация, что также является некорректным.
# Более правильным вариантом в данном случае было бы выделить каждую вакцину в отдельности 
# и посчитать, в скольких странах она используется

#избавляемся от пропусков в столбце с вакцинами
combination_not_nan = covid_df.dropna(subset=['vaccines'])
#группируем по странам и выбираем последнюю комбинацию вакцин в стране
combination_last = combination_not_nan.groupby(['country'])['vaccines'].last() 
#заменяем пробелы в строках и делим комбинации на отдельные вакцины по запятой
combination_last = combination_last.apply(lambda x: x.replace(' ', '').split(',')) 
#создаём пустой словарь, ключами которого будут вакцины, а значениями — количество стран
vaccines_dict = {} 
#создаём цикл по элементам Series combination_last
for vaccines_list in combination_last:
    #создаём цикл по элементам списка с вакцинами
    for vaccine in vaccines_list:
        #проверяем условие, что текущей вакцины ещё нет среди ключей словаря
        if vaccine not in vaccines_dict:#если условие выполняется
            vaccines_dict[vaccine] = 1 #заносим вакцину в словарь со значением 1
        else: #в противном случае
            vaccines_dict[vaccine] += 1 #увеличивает количество
#преобразуем словарь в Series
vaccines = pd.Series(vaccines_dict).sort_values(ascending=False)
#выделяем ТОП-7 вакцин и добавляем категорию 'other'
vaccines['other'] = vaccines[7:].sum()
vaccines = vaccines.sort_values(ascending=False)[:8]

"""
#строим круговую диаграмму
fig = plt.figure(figsize=(5, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.pie(
    vaccines,
    labels=vaccines.index,
    autopct='%.1f%%',
)
"""

#отобразим ТОП-5 стран по общему числу привитых вторым компонентом людей (people_fully_vaccinated), 
# а также ТОП-5 стран по числу полностью привитых на 100 человек населения (people_fully_vaccinated_per_hundred)
vacc_country = covid_df.groupby('country')['people_fully_vaccinated'].last().nlargest(5)
vacc_country_per_hundred = covid_df.groupby('country')['people_fully_vaccinated_per_hundred'].last().nlargest(5)

"""
#визуализация главного графика
fig = plt.figure(figsize=(13, 4))
main_axes = fig.add_axes([0, 0, 1, 1])
main_axes.bar(x = vacc_country.index, height = vacc_country);
main_axes.set_ylabel('Число вакцинированных (2 компонент)')
main_axes.set_title('Топ 5 стран по числу полностью привитых людей')
#визуализация вспомогательного графика
insert_axes = fig.add_axes([0.6, 0.6, 0.38, 0.38])
insert_axes.bar(x = vacc_country_per_hundred.index, height = vacc_country_per_hundred, width=0.5);
insert_axes.set_ylabel('На 100 человек')
insert_axes.xaxis.set_tick_params(rotation=45)
"""
"""
#три графика:
# Столбчатую диаграмму, которая покажет динамику ежедневной вакцинации в России.
# Линейный график изменения ежедневной заболеваемости в стране.
#Гистограмму ежедневной заболеваемости в стране.
russia_data = covid_df[covid_df["country"] == "Russia"]
# визуализация систем координат
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
# столбчатая диаграмма
axes[0].bar(
    x=russia_data["date"],
    height=russia_data["daily_vaccinations"],
    label="Число вакцинированных",
)
axes[0].set_title("Ежедневная вакцинация в России")
axes[0].xaxis.set_tick_params(rotation=45)
# линейный график
axes[1].plot(
    russia_data["date"],
    russia_data["daily_confirmed"],
    label="Число заболевших",
    color="tomato",
    lw=2,
)
axes[1].set_title("Ежедневная заболеваемость в России")
axes[1].xaxis.set_tick_params(rotation=45)
# гистограмма
axes[2].hist(
    x=russia_data["daily_confirmed"], label=["Число заболевших"], color="lime", bins=20
)
axes[2].set_title("Гистограмма заболеваемости в России")
axes[2].xaxis.set_tick_params(rotation=30)
"""

#модульный подход при работе с бибилиотекой Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

"""
#код строит столбчатую диаграмму: по оси x откладываются элементы списка names (названия групп), а высоту столбцов определяет список values.
names = ['group_a', 'group_b', 'group_c']
values = [1, 10, 100]
plt.bar(names, values)
plt.show()

#библиотека seaborn
import seaborn as sns
print(sns.__version__)

countries = ['Russia', 'Australia', 'Germany', 'Canada', 'United Kingdom']
croped_covid_df = covid_df[covid_df['country'].isin(countries)]

populations = pd.DataFrame([
    ['Canada', 37664517],
    ['Germany', 83721496],
    ['Russia', 145975300],
    ['Australia', 25726900],
    ['United Kingdom', 67802690]
    ],
    columns=['country', 'population']
)
croped_covid_df = croped_covid_df.merge(populations, on=['country'])

#croped_covid_df.head()
croped_covid_df['daily_confirmed_per_hundred'] = croped_covid_df['daily_confirmed'] / croped_covid_df['population'] * 100
"""
"""
#две гистограммы: одна будет иллюстрировать общее распределение ежедневной заболеваемости (daily_confirmed), 
# а вторая — то же распределение в разрезе стран. 
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))
sns.histplot(
    data=croped_covid_df,
    x='daily_confirmed_per_hundred',
    bins=25,
    kde=True,
    ax=axes[0]
)
axes[0].set_title('Гистограмма ежедневной заболеваемости на 100 человек', fontsize=16)
sns.histplot(
    data=croped_covid_df,
    x='daily_confirmed_per_hundred',
    y='country',
    bins=25,
    color='red',
    ax=axes[1]
)
"""

"""
#коробчатая диаграмма признака летальности
fig = plt.figure(figsize=(10, 7))
boxplot = sns.boxplot(
    data=croped_covid_df,
    y='country',
    x='death_rate',
    orient='h',
    width=0.9
)
boxplot.set_title('Распределение летальности по странам');
boxplot.set_xlabel('Летальность');
boxplot.set_ylabel('Страна');
boxplot.grid()

#Построим jointplot зависимости ежедневной заболеваемости в странах от общей численности населения в процентах (daily_confirmed_per_hundred)
# от числа полностью вакцинированных в процентах (people_fully_vaccinated_per_hundred)
jointplot = sns.jointplot(
    data=croped_covid_df, 
    x='people_fully_vaccinated_per_hundred', 
    y='daily_confirmed_per_hundred',
    hue='country',
    xlim = (0, 40),
    ylim = (0, 0.1),
    height=8,
)

#тепловая карта для сравнения темпов вакцинации вакцинации по странам во времени
pivot = croped_covid_df.pivot_table(
    values='people_vaccinated_per_hundred',
    columns='date',
    index='country',
)
pivot.columns = pivot.columns.astype('string')
#display(pivot)

heatmap = sns.heatmap(data=pivot, cmap='YlGnBu')
heatmap.set_title('Тепловая карта вакцинации', fontsize=16)
"""

#Создайте новый признак confirmed_per_hundred, который покажет процентное отношение заболевших вирусом к общему числу населения в странах ()
#croped_covid_df['confirmed_per_hundred'] = croped_covid_df['confirmed'] / croped_covid_df['population'] * 100

"""
#Постройте тепловую карту, которая покажет, как росло число заболевших в процентах 
# от общего числа населения (confirmed_per_hundred) в странах из таблицы croped_covid_df
pivot = croped_covid_df.pivot_table(
    values='confirmed_per_hundred',
    columns='date',
    index='country'
)
pivot.columns = pivot.columns.astype('string')
fig = plt.figure(figsize=(10, 5))

heatmap = sns.heatmap(data=pivot, cmap='plasma')
heatmap.set_title('Тепловая карта заболеваемости', fontsize=16)

#Постройте коробчатую диаграмму для признака recover_rate (отношение выздоровлений к числу зафиксированных случаев заболевания в процентах).
fig = plt.figure(figsize=(10, 7))
boxplot = sns.boxplot(
    data=croped_covid_df,
    x='country',
    y='recover_rate',
    orient='v',
    width=0.9
)
"""

#библиотека PLOTLY
import plotly
#import plotly.express as px

#print(plotly.__version__) #5.24.1
"""
#Построим график роста зафиксированного числа случаев заражения (confirmed), 
# смертей (deaths), выздоровлений (recovered) и активных случаев (active) за всё время. 
# Для этого просуммируем статистику по дням и передадим полученный DataFrame в метод line()
line_data = covid_df.groupby('date', as_index=False).sum()
fig = px.line(
    data_frame=line_data, #DataFrame
    x='date', #ось абсцисс
    y=['confirmed', 'recovered', 'deaths', 'active'], #ось ординат
    height=500, #высота
    width=1000, #ширина
    title='Confirmed, Recovered, Deaths, Active cases over Time' #заголовок
)
fig.show()


#построим столбчатую диаграмму, показывающую ТОП-10 стран по среднему проценту выздоравливающих пациентов (recover_rate)
bar_data = covid_df.groupby(
    by='country',
    as_index=False
)[['recover_rate']].mean().round(2).nlargest(10, columns=['recover_rate'])
#строим график
fig = px.bar(
    data_frame=bar_data, #датафрейм
    x="country", #ось x
    y="recover_rate", #ось y
    color='country', #расцветка в зависимости от страны
    text = 'recover_rate', #текст на столбцах
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000, #ширина
    title='Top 10 Countries for Recovery Rate' #заголовок
)

#отображаем его
fig.show()


#Построим иерархическую диаграмму для среднего ежедневного показателя выздоровевших пациентов (daily_recovered) во всех странах.
#считаем среднее ежедневно фиксируемое количество выздоровевших по странам
treemap_data = covid_df.groupby(
    by='country',
    as_index=False
)[['daily_recovered']].mean()
#строим график
fig = px.treemap(
    data_frame=treemap_data, #DataFrame
    path=['country'], #категориальный признак, для которого строится график
    values='daily_recovered', #параметр, который сравнивается
    height=500, #высота
    width=1000, #ширина
    title='Daily Recovered Cases by Country' #заголовок
)
#отображаем график
fig.show()


#построим фоновую картограмму, которая покажет распространение (confirmed) коронавируса в мире во времени
#преобразуем даты в строковый формат
choropleth_data = covid_df.sort_values(by='date')
choropleth_data['date'] = choropleth_data['date'].astype('string')
#строим график
fig = px.choropleth(
    data_frame=choropleth_data, #DataFrame
    locations="country", #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой Plotly
    color="confirmed", #от чего зависит цвет
    animation_frame="date", #анимационный бегунок
    range_color=[0, 30e6], #диапазон цвета
    title='Global Spread of COVID-19', #заголовок
    width=800, #ширина
    height=500, #высота
    color_continuous_scale='Reds' #палитра цветов
)
#отображаем график
fig.show()

#Построим 3D-диаграмму рассеяния, которая покажет, как число ежедневно обнаруживаемых случаев
# и число ежедневных смертей влияют на желание людей вакцинироваться
#фильтруем таблицу по странам
countries=['United States', 'Russia', 'United Kingdom', 'Brazil', 'France']
scatter_data = covid_df[covid_df['country'].isin(countries)]
#строим график
fig = px.scatter_3d(
    data_frame=scatter_data, #DataFrame
    x = 'daily_confirmed', #ось абсцисс
    y = 'daily_deaths', #ось ординат
    z = 'daily_vaccinations', #ось аппликат
    color='country', #расцветка в зависимости от страны
    log_x=True, 
    log_y=True,
    width=1000,
    height=700
)
#отображаем график
fig.show()
fig.write_html("plotly/scatter_3d.html")


#Постройте линейный график, который отображает, как изменялось ежедневное количество вакцинированных (daily_vaccinations) в мире во времени. 
# Из графика найдите, чему равно количество вакцинированных (в миллионах) 28 февраля 2021 года (2021-02-28). 
# Ответ округлите до целого числа: = 6
line_data = covid_df.groupby('date', as_index=False)['daily_vaccinations'].sum()
fig = px.line(data_frame=line_data, x = 'date', y='daily_vaccinations')
fig.show()
""" 
#Постройте анимированную тепловую картограмму для числа поставленных вакцин во всём мире (total_vaccinations). 
# На полученной карте найдите, чему равно количество вакцинированных в Японии (Japan) 
# на 24 марта 2021 года (2021-03-24). Ответ приведите в тысячах (без нулей) и округлите до целого числа: 741 (тыс)
choropleth_data = covid_df.sort_values(by='date')
choropleth_data['date'] = choropleth_data['date'].astype('string')
#строим график
fig = px.choropleth(
    data_frame=choropleth_data, #DataFrame
    locations="country", #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой Plotly
    color="total_vaccinations", #от чего зависит цвет
    hover_name = "country", #группирующая переменная
    animation_frame="date", #анимационный бегунок
    range_color=[0, 600e6], #диапазон цвета
    color_continuous_scale='Reds' #палитра цветов
)
#отображаем график
fig.show()


SyntaxError: incomplete input (541735717.py, line 507)